In [4]:
"""
MFC Primitive Baseline Simulation
=================================
Simple bang-bang control with fixed 5mL doses.
Same priority logic as state-space, but no model - just threshold checks.

This serves as a baseline to compare against the state-space controller.
"""

import numpy as np

# =============================================================================
# SYSTEM PARAMETERS
# =============================================================================

# Reactor volume (mL)
V = 1000.0

# Dose volume (mL) - fixed, same as state-space
DOSE_VOLUME = 5.0

# Simulation time step (minutes)
DT = 1.0

# -----------------------------------------------------------------------------
# Substrate Properties
# -----------------------------------------------------------------------------
SUBSTRATES = {
    'wine': {
        'pH': 4.0,
        'EC': 0.02,
        'TAN': 0.1,
        'Fe': 0.05, 'Zn': 0.01, 'Cu': 0.001, 'Mn': 0.02, 'Se': 0.0, 'Ni': 0.0, 'Co': 0.0
    },
    'urine': {
        'pH': 8.5,
        'EC': 6.25,
        'TAN': 200.0,
        'Fe': 0.01, 'Zn': 0.05, 'Cu': 0.01, 'Mn': 0.005, 'Se': 0.001, 'Ni': 0.002, 'Co': 0.001
    },
    'spirulina': {
        'pH': 9.0,
        'EC': 1.14,
        'TAN': 144.0,
        'Fe': 4.275, 'Zn': 0.300, 'Cu': 0.915, 'Mn': 0.285, 'Se': 0.00105, 'Ni': 0.0, 'Co': 0.0
    }
}

# -----------------------------------------------------------------------------
# Control Targets
# -----------------------------------------------------------------------------
TARGETS = {
    'pH': {'min': 6.8, 'max': 7.2, 'setpoint': 7.0},
    'EC': {'min': 5.0, 'max': 10.0, 'setpoint': 7.5},
    'TAN': {'min': 50.0, 'max': 200.0, 'setpoint': 125.0}
}

# Metal targets (for reporting only - not actively controlled)
METAL_TARGETS = {
    'Fe':  {'min': 1.0, 'max': 10.0},
    'Zn':  {'min': 0.1, 'max': 1.0},
    'Cu':  {'min': 0.01, 'max': 0.5},
    'Mn':  {'min': 0.1, 'max': 1.0},
    'Se':  {'min': 0.01, 'max': 0.1},
    'Ni':  {'min': 0.05, 'max': 1.0},
    'Co':  {'min': 0.01, 'max': 0.3},
}

# -----------------------------------------------------------------------------
# Drift / Decay (same as state-space)
# -----------------------------------------------------------------------------
DRIFT = {
    'pH': -0.0005,   # pH drifts down
    'EC': -0.001,    # EC drifts down
    'TAN': -0.3      # TAN consumed
}

METAL_DECAY_RATE = 0.0005  # per minute


# =============================================================================
# MIXING PHYSICS (CSTR)
# =============================================================================

def mix_state(reactor_state, substrate_name, dose_volume, reactor_volume):
    """
    CSTR mixing: x_new = (x_reactor * V_reactor + x_input * V_dose) / (V_reactor + V_dose)
    """
    substrate = SUBSTRATES[substrate_name]
    new_volume = reactor_volume + dose_volume

    new_state = {}
    for key in reactor_state:
        if key in substrate:
            new_state[key] = (reactor_state[key] * reactor_volume + substrate[key] * dose_volume) / new_volume
        else:
            new_state[key] = reactor_state[key]

    return new_state, new_volume


# =============================================================================
# DRIFT / DECAY
# =============================================================================

def apply_drift(state, dt):
    """
    Apply autonomous dynamics:
    - pH, EC, TAN: constant drift
    - Metals: first-order decay
    """
    new_state = state.copy()

    # Constant drift for pH, EC, TAN
    new_state['pH'] = max(0, state['pH'] + DRIFT['pH'] * dt)
    new_state['EC'] = max(0.01, state['EC'] + DRIFT['EC'] * dt)
    new_state['TAN'] = max(0, state['TAN'] + DRIFT['TAN'] * dt)

    # First-order decay for metals
    decay_factor = np.exp(-METAL_DECAY_RATE * dt)
    for metal in ['Fe', 'Zn', 'Cu', 'Mn', 'Se', 'Ni', 'Co']:
        new_state[metal] = state[metal] * decay_factor

    return new_state


# =============================================================================
# CONTROL LAW (Simple Bang-Bang)
# =============================================================================

def control_decision(state):
    """
    Priority-based bang-bang control.
    Same logic as state-space, but no model prediction.

    Priority order:
    1. pH (highest) - bacterial survival
    2. TAN - nitrogen for growth
    3. EC - affects power output

    Returns: substrate name to dose, or None
    """
    pH = state['pH']
    EC = state['EC']
    TAN = state['TAN']

    # Priority 1: pH control
    if pH > TARGETS['pH']['max']:
        return 'wine', "DOSE WINE (pH high)"
    if pH < TARGETS['pH']['min']:
        return 'urine', "DOSE URINE (pH low)"

    # Priority 2 & 3: TAN and EC combined logic
    if TAN < TARGETS['TAN']['min'] and EC < TARGETS['EC']['min']:
        return 'urine', "DOSE URINE (TAN+EC)"

    if TAN < TARGETS['TAN']['min']:
        return 'spirulina', "DOSE SPIRULINA (TAN)"

    if EC < TARGETS['EC']['min']:
        return 'urine', "DOSE URINE (EC low)"

    # All in range - no dosing needed
    return None, "---"


# =============================================================================
# SIMULATION
# =============================================================================

def run_simulation(duration_minutes=480, initial_state=None):
    """
    Run the primitive baseline simulation.
    """

    # Default initial state
    if initial_state is None:
        initial_state = {
            'pH': 7.0,
            'EC': 5.5,
            'TAN': 80.0,
            'Fe': 1.0,
            'Zn': 0.1,
            'Cu': 0.05,
            'Mn': 0.1,
            'Se': 0.01,
            'Ni': 0.01,
            'Co': 0.01
        }

    # Initialize
    state = initial_state.copy()
    volume = V
    history = []

    # Counters
    total_volumes = {'wine': 0.0, 'urine': 0.0, 'spirulina': 0.0}
    dose_counts = {'wine': 0, 'urine': 0, 'spirulina': 0}

    # Print header
    print("=" * 130)
    print("MFC PRIMITIVE BASELINE SIMULATION")
    print("Control: Simple bang-bang with fixed 5mL doses")
    print("=" * 130)
    print(f"\nDrift: pH {DRIFT['pH']}/min, EC {DRIFT['EC']}/min, TAN {DRIFT['TAN']}/min")
    print(f"Metal decay: {METAL_DECAY_RATE}/min")
    print(f"\nTargets: pH [{TARGETS['pH']['min']}-{TARGETS['pH']['max']}], "
          f"EC [{TARGETS['EC']['min']}-{TARGETS['EC']['max']}] mS/cm, "
          f"TAN [{TARGETS['TAN']['min']}-{TARGETS['TAN']['max']}] mg-N/L")
    print("\n" + "=" * 130)
    print(f"\n{'Time':<6} {'pH':>6} {'EC':>8} {'TAN':>8} {'Vol':>8} {'Action':<24} "
          f"{'Fe':>6} {'Zn':>6} {'Cu':>6} {'Mn':>6} {'Se':>7} {'Ni':>7} {'Co':>7}")
    print(f"{'(min)':<6} {'':>6} {'(mS/cm)':>8} {'(mg-N/L)':>8} {'(mL)':>8} {'':<24} "
          f"{'(mg/L)':>6} {'(mg/L)':>6} {'(mg/L)':>6} {'(mg/L)':>6} {'(mg/L)':>7} {'(mg/L)':>7} {'(mg/L)':>7}")
    print("-" * 130)

    for t in range(int(duration_minutes) + 1):
        # Store history
        history.append({
            'time': t,
            'volume': volume,
            **state.copy()
        })

        # Control decision
        action, action_str = control_decision(state)

        # Print current state
        print(f"{t:<6} {state['pH']:>6.2f} {state['EC']:>8.2f} {state['TAN']:>8.1f} {volume:>8.0f} {action_str:<24} "
              f"{state['Fe']:>6.3f} {state['Zn']:>6.3f} {state['Cu']:>6.3f} {state['Mn']:>6.3f} "
              f"{state['Se']:>7.4f} {state['Ni']:>7.4f} {state['Co']:>7.4f}")

        # Apply control action (fixed 5mL dose)
        if action:
            state, volume = mix_state(state, action, DOSE_VOLUME, volume)
            dose_counts[action] += 1
            total_volumes[action] += DOSE_VOLUME

        # Apply drift for next time step
        if t < duration_minutes:
            state = apply_drift(state, DT)

    # Print summary
    print("-" * 130)
    print("\n" + "=" * 130)
    print("SIMULATION SUMMARY")
    print("=" * 130)

    total_dosed = sum(total_volumes.values())
    print(f"\nDuration: {duration_minutes} minutes ({duration_minutes/60:.1f} hours)")
    print(f"Final Volume: {volume:.0f} mL")

    print(f"\nSubstrate Usage:")
    for sub in ['wine', 'urine', 'spirulina']:
        pct = (total_volumes[sub] / total_dosed * 100) if total_dosed > 0 else 0
        print(f"  {sub.capitalize():<12}: {total_volumes[sub]:>8.0f} mL ({pct:>5.1f}%) - {dose_counts[sub]} doses")
    print(f"  {'Total':<12}: {total_dosed:>8.0f} mL")

    print(f"\nFinal State:")
    print(f"  pH:  {state['pH']:.2f} (target: {TARGETS['pH']['min']}-{TARGETS['pH']['max']})")
    print(f"  EC:  {state['EC']:.2f} mS/cm (target: {TARGETS['EC']['min']}-{TARGETS['EC']['max']})")
    print(f"  TAN: {state['TAN']:.1f} mg-N/L (target: {TARGETS['TAN']['min']}-{TARGETS['TAN']['max']})")

    print(f"\nFinal Trace Metals:")
    for metal in ['Fe', 'Zn', 'Cu', 'Mn', 'Se', 'Ni', 'Co']:
        val = state[metal]
        lo, hi = METAL_TARGETS[metal]['min'], METAL_TARGETS[metal]['max']
        status = "OK" if lo <= val <= hi else ("LOW" if val < lo else "HIGH")
        print(f"  {metal}: {val:.4f} mg/L (target: {lo}-{hi}) {status}")

    return history


# =============================================================================
# MAIN
# =============================================================================

if __name__ == "__main__":
    history = run_simulation(duration_minutes=480)

MFC PRIMITIVE BASELINE SIMULATION
Control: Simple bang-bang with fixed 5mL doses

Drift: pH -0.0005/min, EC -0.001/min, TAN -0.3/min
Metal decay: 0.0005/min

Targets: pH [6.8-7.2], EC [5.0-10.0] mS/cm, TAN [50.0-200.0] mg-N/L


Time       pH       EC      TAN      Vol Action                       Fe     Zn     Cu     Mn      Se      Ni      Co
(min)          (mS/cm) (mg-N/L)     (mL)                          (mg/L) (mg/L) (mg/L) (mg/L)  (mg/L)  (mg/L)  (mg/L)
----------------------------------------------------------------------------------------------------------------------------------
0        7.00     5.50     80.0     1000 ---                       1.000  0.100  0.050  0.100  0.0100  0.0100  0.0100
1        7.00     5.50     79.7     1000 ---                       1.000  0.100  0.050  0.100  0.0100  0.0100  0.0100
2        7.00     5.50     79.4     1000 ---                       0.999  0.100  0.050  0.100  0.0100  0.0100  0.0100
3        7.00     5.50     79.1     1000 ---       